In [16]:
import pandas as pd
import re

In [ ]:
REPO_ROOT = "../"

In [6]:
train = pd.read_csv(REPO_ROOT + "Data/train.csv")
test = pd.read_csv(REPO_ROOT + "Data/test.csv")

In [12]:
train.head()

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,2,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1


In [8]:
train.columns

Index(['EditType', 'EditID', 'comment', 'user', 'user_edit_count',
       'user_distinct_pages', 'user_warns', 'user_reg_time', 'prev_user',
       'common', 'current', 'previous', 'page_made_time', 'title', 'namespace',
       'creator', 'num_recent_edits', 'num_recent_reversions', 'current_minor',
       'current_timestamp', 'added_lines', 'previous_timestamp',
       'deleted_lines', 'isvandalism', 'num_edits_5d_before', 'is_person'],
      dtype='object')

In [ ]:
dumbIPv4Regex = re.compile(r"^[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}$")
# Compiles a regex object matching any string of the form "xxx.xxx.xxx.xxx",
# where "xxx" is a set of between 1 and 3 decimal digits.
# No need to do any more validation,
# since the dataset only contains valid IPs (users are not allowed to have IP-like usernames)

In [44]:
if "user_is_IP" not in pd.concat([train, test]).columns:
    train["user_is_IP"] = train["user"].apply(
        lambda x: bool(dumbIPv4Regex.fullmatch(x))
    )
    test["user_is_IP"] = test["user"].apply(lambda x: bool(dumbIPv4Regex.fullmatch(x)))

In [ ]:
# Empty strings got recorded as 'nan', which is inferred as float.
# Replace with empty string again so that applying len() makes sense
# TODO: find why empty strings got recorded as 'nan'
# and fix the problem at its source
train["added_lines"] = train["added_lines"].replace(float("nan"), "")
test["added_lines"] = test["added_lines"].replace(float("nan"), "")
train["deleted_lines"] = train["deleted_lines"].replace(float("nan"), "")
test["deleted_lines"] = test["deleted_lines"].replace(float("nan"), "")

In [ ]:
if "added_raw_length" not in pd.concat([train, test]).columns:
    train["added_raw_length"] = train["added_lines"].apply(len)
    test["added_raw_length"] = test["added_raw_length"].apply(len)

if "deleted_raw_length" not in pd.concat([train, test]).columns:
    train["deleted_raw_length"] = train["deleted_lines"].apply(len)
    test["deleted_raw_length"] = test["deleted_lines"].apply(len)

In [57]:
if "added_word_count" not in pd.concat([train, test]).columns:
    train["added_word_count"] = train["added_lines"].apply(lambda x: len(x.split()))
    test["added_word_count"] = test["added_lines"].apply(lambda x: len(x.split()))
if "deleted_word_count" not in pd.concat([train, test]).columns:
    train["deleted_word_count"] = train["deleted_lines"].apply(lambda x: len(x.split()))
    test["deleted_word_count"] = test["deleted_lines"].apply(lambda x: len(x.split()))